In [2]:
#imports
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from math import sqrt
import pandas as pd
import glob
from numpy import random
import sys
import pathlib
import math
from math import *
import time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
#manual data exploration
data = pd.read_csv('data/00_tracks.csv')
meta = pd.read_csv('data/00_recordingMeta.csv')
trackmeta = pd.read_csv('data/00_tracksMeta.csv')
data = data.drop(columns=['frame'])
data_lifetime = data.groupby(['trackId','recordingId']).count()
data = data.groupby(['trackId','recordingId']).mean()

frame_rate = float(meta['frameRate'])
data['frame_rate'] = frame_rate
trackmeta_class = trackmeta['class'].to_numpy() 
data['class']= trackmeta_class
#a = data_lifetime['trackLifetime'].to_numpy()
data['num_frames'] = data_lifetime['trackLifetime']
data['time'] = data['num_frames']/data['frame_rate']

data

,,trackLifetime,xCenter,yCenter,heading,width,length,xVelocity,yVelocity,xAcceleration,yAcceleration,lonVelocity,latVelocity,lonAcceleration,latAcceleration,frame_rate,class,num_frames,time
trackId,recordingId,,,,,,,,,,,,,,,,,,
0,0,248.5,136.936457,-124.605041,105.079742,0.8158,1.0443,-0.396327,1.465573,-0.010325,0.040120,1.519606,-0.002443,0.041408,0.002191,25.0,pedestrian,498,19.92
1,0,298.5,87.026364,-71.841533,239.433657,0.5949,0.8228,-0.019571,-0.530214,-0.097151,-0.043559,1.153256,0.012845,-0.019199,0.166041,25.0,pedestrian,598,23.92
2,0,323.0,134.713267,-116.205629,103.798116,0.6776,1.2047,-0.422732,1.708808,0.000329,0.006917,1.762987,-0.003229,0.006277,0.002071,25.0,pedestrian,647,25.88
3,0,144.0,101.887958,-96.417955,309.427871,0.8368,1.8363,3.473714,-4.035116,-0.358681,-0.336022,6.050080,-0.018958,-0.064207,0.624173,25.0,bicycle,289,11.56
4,0,124.5,102.200078,-96.155527,310.023437,0.8188,1.8237,3.994376,-4.585387,-0.547654,-0.486018,6.933116,-0.043660,-0.096362,0.837646,25.0,bicycle,250,10.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,0,12858.0,129.553500,-109.255500,101.649800,1.9075,4.5796,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,25.0,car,25717,1028.68
344,0,12858.0,131.107700,-115.328900,102.641100,2.0268,5.5257,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,25.0,car,25717,1028.68
345,0,12858.0,133.378900,-124.188800,106.300900,1.8057,4.7132,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,25.0,car,25717,1028.68


In [4]:
meta

,recordingId,locationId,frameRate,speedLimit,weekday,startTime,duration,numTracks,numVehicles,numVRUs,latLocation,lonLocation,xUtmOrigin,yUtmOrigin,orthoPxToMeter
0,0,1,25,13.8889,tuesday,7,1028.64,348,324,24,50.7906,6.0599,292669.4681,5630731.704,0.01481


In [5]:
#find avg vehicle size
trackmeta = pd.read_csv('data/23_tracksMeta.csv')
trackmeta['size']=trackmeta['width']*trackmeta['length']
track = trackmeta.groupby('class').min()

In [6]:
trackmeta

,recordingId,trackId,initialFrame,finalFrame,numFrames,width,length,class,size
0,23,0,19212,19783,572,0.8494,1.5849,bicycle,1.346214
1,23,1,17904,18125,222,0.9753,1.8031,motorcycle,1.758563
2,23,2,15933,16446,514,0.7872,1.8639,motorcycle,1.467262
3,23,3,13575,14323,749,0.8861,1.3904,bicycle,1.232033
4,23,4,12769,13865,1097,0.7866,1.2694,bicycle,0.998510
...,...,...,...,...,...,...,...,...,...
459,23,459,23365,23654,290,2.2057,5.1967,car,11.462361
460,23,460,23502,23715,214,3.5310,12.5956,truck,44.475064
461,23,461,23519,23716,198,2.1717,5.2090,car,11.312385
462,23,462,23571,23716,146,2.0845,4.9813,car,10.383520
